In [203]:
from bokeh.models import Legend
from bokeh.layouts import layout
from bokeh.palettes import Spectral6, OrRd3, GnBu3
from bokeh.transform import linear_cmap
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, BoxSelectTool, BoxZoomTool, LassoSelectTool, ResetTool, PanTool
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, CustomJS, Slider, Range, Label, LabelSet, Range1d
from bokeh.models import Div, RangeSlider, Spinner

# from bokeh.models import MultiChoice

# OPTIONS = ["foo", "bar", "baz", "quux"]

# multi_choice = MultiChoice(value=["foo", "baz"], options=OPTIONS)
# multi_choice.js_on_change("value", CustomJS(code="""
#     console.log('multi_choice: value=' + this.value, this.toString())
# """))

# show(multi_choice)

def plot_chr(df_chr):
    output_notebook()
    df = df_chr.copy()
    df['chr_int'] = pd.to_numeric(df['chr'].str.lstrip('chr'), errors='coerce')
    df.loc[df['chr_int'].isna(), 'chr_int'] = df.loc[df['chr_int'].isna(), 'chr'].str.lstrip('chr')
    df = df.sort_values(by=['chr_int', 'Position'], ascending=True)
    
    
    p = figure(y_range=(0, 5), width=1600, height=300, 
               tools=[BoxZoomTool(), ResetTool(), PanTool(dimensions="width"), HoverTool()],
              )
    legend_item = []
    
    
    mapper1 = linear_cmap(field_name='copyNum', palette=OrRd3+ tuple(reversed(OrRd3)), 
                          low=1 ,high=3)
    mapper2 = linear_cmap(field_name='copyNum', palette=GnBu3 + tuple(reversed(GnBu3)),
                          low=1 ,high=3)
    spinner = Spinner(
        title="Circle size",  # a string to display above the widget
        low=0.5,  # the lowest possible number to pick
        high=10,  # the highest possible number to pick
        step=0.5,  # the increments by which the number can be adjusted
        value=3,  # the initial value to display in the widget
        width=200,  #  the width of the widget in pixels
    )
    
    chrs_int = df['chr_int'].unique()
    chrs = df['chr'].unique()
    end_point = 0
    df['Position_axis'] = end_point
    for c, ch in zip(chrs_int, chrs):
        # ch = f'chr{c}'
        # if  (c == 'X') or (c % 2):
        if type(c) == float and int(c) % 2:
            mapper = mapper1
        elif c == 'X':
            mapper = mapper1
        else:
            mapper = mapper2
            
        # if ch  in df['chr'].unique():
        df.loc[df['chr']==ch, 'Position_axis'] = df.loc[df['chr']==ch, 'Position'] + end_point
        end_point = df['Position_axis'].max()
        data = ColumnDataSource(df.loc[df['chr']==ch])
        pc = p.circle(x='Position_axis', y='copyNum', color=mapper, fill_alpha=0.5, size=3, line_width=0, source=data)#, legend_label=c)
        # legend_item.append((ch, [pc]))
        spinner.js_link("value", pc.glyph, "size")

    # legend = Legend(items=legend_item, location="center", orientation = "horizontal", click_policy="hide")

    p.x_range.update_from_json({'start':0, 'end':end_point})
    range_slider = RangeSlider(
        width=1600,
        show_value=False,
        title="Adjust x-axis range", # a title to display above the slider
        start=0,  # set the minimum value for the slider
        end=end_point,  # set the maximum value for the slider
        step=1,  # increments for the slider
        value=(0, end_point),  # initial values for slider
    )
    range_slider.js_link("value", p.x_range, "start", attr_selector=0)
    range_slider.js_link("value", p.x_range, "end", attr_selector=1)
    
    df_first_pos = df.drop_duplicates(subset='chr_int', keep='first')
    data = ColumnDataSource(df_first_pos)
    labels = LabelSet(x='Position_axis', y=4, text='chr',
              x_offset=5, y_offset=10, source=data, render_mode='canvas')
    
    p.toolbar.logo = None
    p.add_layout(labels)
    # p.add_layout(legend, 'above')    
    plot = layout([[spinner],[range_slider],[p],])
    # plot = layout([[spinner],[p],])
    show(plot)


In [194]:
df['chr_int'] = pd.to_numeric(df['chr'].str.lstrip('chr'), errors='coerce')
df.loc[df['chr_int'].isna(), 'chr_int'] = df.loc[df['chr_int'].isna(), 'chr'].str.lstrip('chr')
df = df.sort_values(by=['chr_int', 'Position'], ascending=True)

df_first_pos = df.drop_duplicates(subset='chr_int', keep='first')

In [195]:
df_first_pos

,chr,Position,copyNum,chr_int
0,chr1,4350001,2.165106,1.0
2247,chr2,150001,1.419974,2.0
2988,chr3,450001,1.643338,3.0
3406,chr4,1050001,1.915483,4.0
3803,chr5,1800001,1.849588,5.0
4175,chr6,900001,1.773396,6.0
4533,chr7,1800001,1.758279,7.0
4824,chr8,900001,1.768808,8.0
5123,chr9,450001,1.890266,9.0
446,chr10,600001,1.779802,10.0


In [138]:
df.drop_duplicates(subset='chr', keep='first')

,chr,Position,copyNum
0,chr1,4350001,2.165106
446,chr10,600001,1.779802
706,chr11,3600001,1.523524
961,chr12,300001,1.907495
1233,chr13,20400001,1.891126
1436,chr14,20850001,2.140348
1620,chr15,22950001,2.421781
1776,chr16,3450001,0.998244
1909,chr17,600001,1.682901
2037,chr18,300001,2.041004


In [204]:
import pandas as pd


file = '/mnt/d/Yangk/CNV/PGTA20220413/PGTA_ZHT06.fq_merge_bin_correct_da.csv'
df = pd.read_csv(file)
p = plot_chr(df)

Loading BokehJS ...